In [1]:
import torch
import torch.nn as nn
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

In [2]:
# prepare data
bc = datasets.load_breast_cancer()

In [3]:
X, y = bc.data, bc.target
n_samples, n_features = X.shape
print(X.shape) 
# rows = samples, columns = features

(569, 30)


In [4]:
# train test split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [5]:
# scaling features
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [6]:
# convert to tensors
X_train = torch.from_numpy(X_train.astype(np.float32))
X_test = torch.from_numpy(X_test.astype(np.float32))
y_train = torch.from_numpy(y_train.astype(np.float32))
y_test = torch.from_numpy(y_test.astype(np.float32))

In [7]:
print(y_train.shape)
print(y_train[0])

torch.Size([455])
tensor(1.)


In [8]:
y_train = y_train.view(y_train.shape[0], 1)
y_test = y_test.view(y_test.shape[0], 1)

In [9]:
class LogisticRegression(nn.Module):
    def __init__(self, input_dim):
        super(LogisticRegression, self).__init__()
        self.linear = nn.Linear(input_dim, 1)
        
    def forward(self, x):
        y_hat = torch.sigmoid(self.linear(x))
        return y_hat


model = LogisticRegression(n_features)

In [10]:
loss = nn.BCELoss()
optimizer = torch.optim.SGD(model.parameters(), lr=0.01)

In [11]:
num_epochs = 100

for epoch in range(num_epochs):
    
    # forward pass and loss
    y_hat = model(X_train)
    l = loss(y_hat, y_train)
    
    # backward pass
    l.backward()
    
    # update weights
    optimizer.step()
    
    # zero gradients
    optimizer.zero_grad()
    
    if (epoch+1) % 10 == 0:
        print(f'epoch: {epoch+1}, loss = {l.item()}')

epoch: 10, loss = 0.7353194952011108
epoch: 20, loss = 0.5992152690887451
epoch: 30, loss = 0.5123891234397888
epoch: 40, loss = 0.4526967406272888
epoch: 50, loss = 0.40901637077331543
epoch: 60, loss = 0.37549757957458496
epoch: 70, loss = 0.3488333225250244
epoch: 80, loss = 0.3270227313041687
epoch: 90, loss = 0.30878427624702454
epoch: 100, loss = 0.29325810074806213


In [12]:
with torch.no_grad():
    y_pred = model(X_test)
    y_pred_cls = y_pred.round()
    acc = y_pred_cls.eq(y_test).sum() / float(y_test.shape[0])
    print(f'accuracy = {acc:.4f}')

accuracy = 0.9649
